# 大模型路由映射传递数据

RunnablePassthrough 允许不改变或添加额外的键来传递输入。这通常与 RunnableParallel 结合使用，将数据分配给映射中的新键。

RunnablePassthrough() 单独调用，将简单地获取输入并将其传递。

使用分配 ( RunnablePassthrough.assign(...) ) 调用的 RunnablePassthrough 将获取输入，并将添加传递给分配函数的额外参数。

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified = lambda x: x["num"] +1
)

runnable.invoke({"num": 2})


{'passed': {'num': 2}, 'extra': {'num': 2, 'mult': 6}, 'modified': 3}

如上所示， passed 键是用 RunnablePassthrough() 调用的，因此它只是传递 {'num': 2} 。

在第二行中，我们使用 RunnablePastshrough.assign 和一个将数值乘以 3 的 lambda。在这种情况下， extra 设置为 {'num': 2, 'mult': 6} ，这是原始的添加了 mult 键的值。

最后，我们还在映射中使用 modified 设置了第三个键，它使用 lambda 来设置单个值，在 num 上加 1，从而得到 modified 键，其值为3 。

## 运行自定义函数

您可以在管道中使用任意函数。

请注意，这些函数的所有输入都必须是单个参数。如果您有一个接受多个参数的函数，则应该编写一个接受单个输入并将其解包为多个参数的包装器。

In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOACL_API_KEY")
api_base = os.getenv("LOACL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=api_base, temperature=0.3)

In [6]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


def lenght_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("What is {a} + {b}?")

chain1 = prompt | model

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(lenght_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
    }
    | chain1
)

chain.invoke({"foo": "hello", "bar": "world"})



AIMessage(content='5 + 25 equals 30.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f28cf0b2-0ba9-40b2-b53d-243816e81ffd-0', usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16, 'input_token_details': {}, 'output_token_details': {}})

## 根据输入动态路由选择逻辑


路由允许您创建非确定性链，其中上一步的输出定义下一步。路由有助于提供与 LLMs 交互的结构和一致性。

执行路由的方法有两种：

1. 有条件地从 RunnableLambda 返回可运行对象（推荐）
2. 使用 RunnableBranch 

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """鉴于下面用户的问题，将其分类为“langchain”，“OpenAI”或“其他”。
    不要用超过一个字赖回答。
    
    <qustion>
    {question}
    </question>
    
    分类："""
)

chain = prompt | model | StrOutputParser()

chain.invoke({"question": "What is the best way to learn Langchain?"})

'langchain'

In [8]:
langchainPrompt = PromptTemplate.from_template(
    """你是langchain方面的专家。
    回答问题时始终以“正如上帝告诉我的那样”开头。
    回答以下问题：
    
    问题：{question}
    
    回答："""
)

langchain_chain = langchainPrompt | model

In [9]:
openaiPrompt = PromptTemplate.from_template(
    """你是OpenAI方面的专家。
    回答问题时始终以“正如奥特曼告诉我的那样”开头。
    回答以下问题：
    
    问题：{question}
    
    回答："""
)

openai_chain = openaiPrompt | model

In [10]:
generalPrompt = PromptTemplate.from_template(
    """回答下列问题：
    问题：{question}
    
    回答："""
)

general_chain = generalPrompt | model


In [11]:
def router(info):
    if "OpenAI" in info["topic"]:
        return openai_chain
    elif "langchain" in info["topic"]:
        return langchain_chain
    else:
        return general_chain


In [12]:
from langchain_core.runnables import RunnableLambda

full_chain = {'topic': chain, 'question': lambda x: x["question"]} | RunnableLambda(router) 

In [14]:
print(full_chain.invoke({"question": "我该如何使用langchain调用OpenAI模型"}))

content='正如上帝告诉我的那样，使用LangChain调用OpenAI模型是一个相对直接的过程。LangChain是一个强大的框架，旨在帮助开发者更轻松地构建基于语言模型的应用。要使用LangChain与OpenAI的API进行交互，您需要按照以下步骤操作：\n\n### 1. 安装必要的库\n\n首先，确保安装了`langchain`和`openai`库。您可以通过pip来安装它们：\n\n```bash\npip install langchain openai\n```\n\n### 2. 获取OpenAI API密钥\n\n您需要从OpenAI获取API密钥。登录到OpenAI的官网（https://platform.openai.com/）并生成您的API密钥。请确保将其保存在一个安全的位置，因为它将用于身份验证。\n\n### 3. 设置OpenAI API密钥\n\n在您的代码中，您需要设置OpenAI的API密钥。可以使用以下方法进行设置：\n\n```python\nimport openai\nopenai.api_key = "your-api-key-here"\n```\n\n为了更好的安全性，建议将密钥存储在环境变量中，避免硬编码在代码中。\n\n### 4. 使用LangChain调用OpenAI模型\n\nLangChain提供了一个`OpenAI`类，它可以非常容易地与OpenAI模型交互。下面是如何设置并使用LangChain调用OpenAI模型的步骤：\n\n```python\nfrom langchain.llms import OpenAI\n\n# 设置OpenAI的API密钥（如果没有在环境变量中设置）\nimport openai\nopenai.api_key = "your-api-key-here"\n\n# 实例化一个OpenAI模型\nllm = OpenAI(model="text-davinci-003", temperature=0.7)  # 你可以选择其他模型，比如 "gpt-3.5-turbo"\n\n# 使用模型生成文本\nresponse = llm("你好，世界！")  # 输入你想问的问题或提示\n\n# 输出模型的回答\nprint(response)\n```\n\n### 